# Ejercicio: clasificando dígitos con redes densas

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/mnist.jpeg" style="width:480px;">

En este ejercicio vamos a tratar de identificar imágenes de dígitos escritos a mano. Usaremos este problema como un campo de pruebas para utilizar diferentes arquitecturas de red neuronal.

## Guía general

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

También vamos a fijar las semillas aleatorias de numpy y tensorflow para obtener resultados reproducibles entre varias ejecuciones del notebook

In [ ]:
import numpy as np
import tensorflow as tf
np.random.seed(1)
tf.random.set_seed(2)

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Carga de datos

El dataset de reconocimiento de dígitos que vamos a trabajar ya está incluído en la librería Keras. Para cargalo solo necesitamos ejecutar lo siguiente

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Las variables **X** que se han cargado están formadas por los dígitos manuscritos a clasificar, mientras que las variables **y** nos indican las etiquetas de las imágenes correspondientes en X. Usaremos los datos de **train** para entrenar nuestra red neuronal, y los datos de **test** para medir el rendimiento de la red.

Podemos comprobar cuántas imágenes tenemos para entrenar y testear de la siguiente manera:

In [ ]:
len(X_train)

In [ ]:
len(X_test)

También podemos comprobar la forma (ancho y alto en píxeles) de una imagen:

In [ ]:
X_test[0].shape

Podemos visualizar las imágenes del dataset a través de la librería **matplotlib**. En la siguiente celda tomamos la primera imagen de entrenamiento y la visualicemos en una escala de grises. También estamos imprimiendo la clase correspondiente a esa imagen, para comprobar que el etiquetado de la imagen es correcto.

In [ ]:
plt.imshow(X_train[0], 'gray')
print("Digit class:", y_train[0])

## Preparación de datos

### Normalización de las entradas

Ante de construir una red neuronal, primero debemos siempre normalizar los datos. Generalmente la normalización implica restar la media y dividir la desviación estándar de los datos. No obstante, para el caso de imágenes en escala de gris como estas, cada valor de los datos representa la intensidad de un pixel, un valor que está acotado en el rango [0, 255]. Podemos por tanto realizar una normalización sencilla consistente en dividir los datos por 255 para que se queden ajustados al rango [0, 1]. También necesitaremos tranformar el tipo de datos a `float`, o de otro modo no podremos representar valores decimales.

In [ ]:
X_train_norm = X_train.astype('float32') / 255

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Realiza la misma normalización para los datos de entrada de test.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

### Codificación de las salidas

Los datos de salida para una red neuronal no requieren normalización, pero sí deben ser codificados siguiendo un formato en particular. En lugar de tener un valor entero en el rango [0, 9] que indique la clase de la imagen, utilizaremos una codificación de tipo <a href="https://en.wikipedia.org/wiki/One-hot">one-hot</a>.

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_encoded = to_categorical(y_train, 10) # We have 10 classes to codify

Podemos comprobar que la transformación ha generado una codificación one-hot correcta

In [ ]:
y_train_encoded

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Realiza la misma codificación para los datos de salida de test.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

### Aplanado (flattening) de las entradas

Nuestra preparación de datos está casi terminada, pero falta un pequeño detalle: nuestros datos son imágenes bidimensionales, pero una red neuronal estándar solo es capaz de trabajar con datos en la forma de vectores unidimensionales de variables. Debemos transformar los datos a vectores antes de introducirlos en la red neuronal, algo que podemos hacer con el método `reshape`. Dado que tenemos 60000 imágenes de entrenamiento de 28x28 (784 en total) píxeles, el `reshape` a realizar es:

In [ ]:
trainvectors = X_train_norm.reshape(60000, 784)

Podemos comprobar ahora que nuestros datos de entrenamiento se han convertido en una matriz de 60000 datos (filas) y 784 variables (columnas).

In [ ]:
trainvectors.shape

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Realiza una transformación similar para los datos de entrada de test (X_test), guardando los datos tras el reshape en una nueva varible llamada <b>testvectors</b>. Ten en cuenta que en test contamos con 10000 imágenes, en lugar de las 60000 de entrenamiento.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

Ten en mente que este aplanado en vectores unidimensionales es algo que solo necesitamos hacer para el tipo de redes que trabajaremos en este notebook. Cuando en otros notebooks avancemos a redes especializadas en imágenes, podremos hacer que la red utilice directamente las imágenes como entradas.

## Perceptrón

Empezaremos tratando el problema con la red neuronal más sencilla: un perceptrón. Esto quiere decir que tendremos una red neuronal sin capas ocultas, únicamente conexiones desde las entradas a las salidas.

### Definiendo la red

Para construir una red en Keras primero debemos definir el tipo de arquitectura:

* **Sequential**: cada nueva capa se conecta a la capa declarada inmediatamente antes en la red, siguiendo una cadena.
* **Functional**: cada capa puede conectarse a la salida de cualquier otra capa declarada antes en la red, siempre y cuando no se formen ciclos.

Para este ejercicio será suficiente con una arquitectura de tipo Sequential.

In [ ]:
from tensorflow.keras.models import Sequential
perceptron = Sequential()

Una vez la red ha sido inicializada, podemos ir añadiendo las capas deseadas de manera iterativa. Para construir un perceptrón solo nos hará falta una capa "clásica" de pesos desde las entradas a la salida. En Deep Learning moderno este tipo de capas se llaman **Dense**, porque conectan todas las variables de entrada con todas las salidas de la capa.

In [ ]:
from tensorflow.keras.layers import Dense

Para crear una capa densa suele bastar con declarar el número de unidades de salida (o desde un punto de vista clásico, el número de neuronas). No obstante, en un modelo de tipo Sequential estamos obligados a declarar el número de variables de entrada a la red neuronal cuando creamos nuestra primera capa. En este problema tenemos 784 variables explicativas (28x28 píxeles). En cuanto a las salidas, en este problema tenemos 10 clases, por lo que deberemos crear 10 unidades de salida. También debemos tener en consideración que estamos trabajando un problema multiclase, y por tanto debemos escoger una función de activación que delimite los valores de salida de la red al rango [0, 1], asegurando que la suma de todos estos valores es `1`. La activación `softmax` es la adecuada en estos casos.

In [ ]:
perceptron.add(Dense(10, input_dim=784, activation="softmax"))

Con esto, la definición de la red está completa. Podemos confirmar que la hemos construído correctamente pidiendo a Keras un resumen del modelo.

In [ ]:
perceptron.summary()

### Compilando la red

Tras definir la red, debemos realizar la compilación de la misma. La compilación es un proceso automático que transforma la definición de la red en una formulación simbólica equivalente para la que pueden calcularse las derivadas, permitiendo así ejecutar el algoritmo de retropropagación (backpropagation). Los únicos parámetros que debemos especificar son la función de pérdida o error que la red debe minimizar, y el optimizador a usar durante el aprendizaje.

Dado que estamos tratando con un problema de clasificación multiclase, la función de pérdida más adecuada es la **categorical crossentropy**. En cuanto al optimizador, de momento utilizaremos el **Stochastic Gradient Descent**. Como parámetro opcional, solicitaremos que como métrica se nos informe de la **accuracy** durante el entrenamiento de la red.

In [ ]:
perceptron.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Entrenando la red

Ahora podemos invocar al método `fit` de la red, el encargado de ejecutar el proceso de entrenamiento. Para este problema utilizaremos un tamaño de batch de 128, y 20 épocas de entrenamiento.

In [ ]:
perceptron.fit(
    trainvectors, # Training data
    y_train_encoded, # Labels of training data
    batch_size=128, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    verbose=2 # Level of verbosity of the log messages
)

Ahora que nuestra red está entrenada, podemos obtener predicciones para el conjunto de test como

In [ ]:
probs = perceptron.predict(testvectors)

Las predicciones se obtienen como una matriz con forma `(n_datos, clases)`, que pueden interpretarse como las probabilidades de que la imagen pertenezca a cada una de las clases posibles.

In [ ]:
probs

Si usando estas probabilidades tuviéramos que apostar por qué digito es el que se representa en la imagen, lo más sensato sería decantarnos por el que tiene mayor probabilidad. Podemos hacer esto fácilmente empleando la función [argmax de numpy](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html):

In [ ]:
import numpy as np

preds = np.argmax(probs, axis=-1)
preds

Ahora que tenemos predicciones, si por ejemplo tomamos el primer ejemplo de test, podemos ver su imagen y la correspondiente clase predicha por la red

In [ ]:
plt.imshow(X_test[0], 'gray')
print("Real class", y_test[0], "predicted class", preds[0])

Puede que para este caso particular hayamos acertado pero, ¿qué ocurre con el resto del dataset de test? Una forma rápida de encontrar todas las imágenes de test en las que hemos fallado es usando la función [where](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.where.html) de numpy, que nos devuelve las posiciones en las que una cierta lista contiene el valor `True`. Para este caso, construimos una lista de los índices de las imágenes en las que las predicciones no son correctas, comparando la lista de predicciones con la lista de clases reales de los dígitos de test:

In [ ]:
(fails,) = np.where(y_test != preds)
fails

Vamos a visualizar alguna de las imágenes en las que la red se ha equivocado:

In [ ]:
plt.imshow(X_test[fails[0]], 'gray')
print("Real class", y_test[fails[0]], "predicted class", preds[fails[0]])

Podríamos probar cualquier otra imagen fallida del conjunto de test para analizar en qué tipo de imágenes nuestra red está cometiendo errores. Pero si queremos tener una idea más general de cómo de bien está funcionando la red, podemos calcular su acierto (accuracy). Esto se hace con el método `evaluate` de la red neuronal.

In [ ]:
score = perceptron.evaluate(testvectors, y_test_encoded)
print("Test loss", score[0])
print("Test accuracy", score[1])

Ahora podríamos plantearnos, ¿es este nivel de acierto suficientemente bueno? Imaginemos que nuestro detector de dígitos se usa para escanear los códigos postales de envío de un paquete. Si consideramos que los [códigos postales extendidos de los Estados Unidos](https://en.wikipedia.org/wiki/ZIP_Code) están conformados por 9 dígitos, y que con fallar uno de los dígitos estaríamos enviando un paquete al lugar equivocado... ¿cuál es la probabilidad de mandar el paquete correctamente?

Calcular esta probabilidad es sencillo. Sabemos la probabilidad de que nuestro modelo haga una buena predicción para un solo dígito (la accuracy), así que solo debemos calcular la probabilidad de que el modelo acierte a la vez para los 9 dígitos del código postal. Esta probabilidad conjunta se calcularía como $P(acierto) * P(acierto) * P(acierto) ... = P(acierto)^9$.

In [ ]:
score[1]**9

Podemos concluir que usar un perceptrón en este problema tendría resultados catastróficos: solo un 41% de paquetes enviados a la dirección correcta. ¡Necesitamos hacerlo mejor!

## Perceptrón multicapa

Pasando a los años 80, podemos mejorar nuestra red introduciendo **capas ocultas**. En Keras esto se implementa de forma sencilla introduciendo capas Dense adicionales. Por ejemplo, podemos crear una red con una capa oculta de 32 neuronas de la siguiente manera.

In [ ]:
mlp = Sequential()
mlp.add(Dense(32, input_dim=784, activation="sigmoid"))
mlp.add(Dense(10, activation="softmax"))

La red quedaría, por tanto, configurada así

In [ ]:
mlp.summary()

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila el perceptrón multicapa y entrénalo con los datos de train. ¿Qué nivel de acierto se obtiene al evaluarlo en test? ¿Has mejorado respecto del perceptrón anterior?
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE

### Mejorando el diseño de la red

Para mejorar el rendimiento del perceptrón multicapa utilizaremos las siguientes técnicas:
* Mayor número de unidades ocultas
* Mejor función de activación: ReLU
* Mejor optimizador: adam

Definiremos por tanto la red de la seguiente manera:

In [ ]:
mlp_fine = Sequential()
mlp_fine.add(Dense(100, input_dim=784, activation="relu"))
mlp_fine.add(Dense(10, activation="softmax"))

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Compila la red definida arriba, escogiendo "adam" como algoritmo de optimización, y entrénala con los datos de train. Mide el rendimiento sobre los datos de test. ¿Han ayudado estos cambios a mejorar el acierto?
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

### Más capas

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Define una nueva red con 2 capas ocultas, cada una de ellas de 512 unidades ocultas con activación ReLU. Para la capa de salida, recuerda utilizar la activación softmax. Compila la red definida, escogiendo "adam" como algoritmo de optimización, y entrénala con los datos de entrenamiento. Mide entonces el rendimiento de esta red sobre los datos de test. ¿Has obtenido mejoras?
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

### Controlando el sobreajuste

Los métodos de **regularización** pueden ayudar a mejorar el rendimiento de una red neuronal, especialmente si el número de parámetros de la red es muy grande y esto nos lleva a tener sobreajuste. Uno de los métodos de regularización más simples y efectivos es el **dropout**. En Keras, el dropout se utiliza como una capa más, denominada `Dropout`, la cual anula aleatoriamente parte las salidas producidas por la capa anterior, reemplazando sus valores por $0$.

Por ejemplo, para crear una red con una capa oculta de un 30% de probabilidad de dropout hacemos lo siguiente:

In [ ]:
from tensorflow.keras.layers import Dropout
sample_network = Sequential()
sample_network.add(Dense(512, input_dim=784, activation="relu"))
sample_network.add(Dropout(0.3))
sample_network.add(Dense(10, activation="softmax"))

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
   Define una nueva red con 2 capas ocultas, cada una de ellas de 512 unidades ocultas con actuvación ReLU. Tras ambas capas ocultas, añade una capa Dropout del 40%. Para la capa de salida, recuerda utilizar la activación softmax. Compila la red definida, escogiendo "adam" como algoritmo de optimización, y entrénala con los datos de entrenamiento. Mide entonces el rendimiento de esta red sobre los datos de test. ¿Ha ayudado el dropout?
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    Intenta crear una red con más capas ocultas. ¿Consigues mejorar el rendimiento en test de esta manera?
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE